In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas import read_csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/audio-interspeech/2075_concatenate_dm_4featu.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df.shape

(2075, 181)

In [5]:
np.random.seed(42)
siz=415
df_read = df.copy()
df1 = df_read.sample(siz)
df_read = df_read.drop(df1.index)
df2 = df_read.sample(siz)
df_read = df_read.drop(df2.index)
df3 = df_read.sample(siz)
df_read = df_read.drop(df3.index)
df4 = df_read.sample(siz)
df_read = df_read.drop(df4.index)
df5 = df_read.copy()

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(415, 181)
(415, 181)
(415, 181)
(415, 181)
(415, 181)


In [6]:
q = list(df1.index)+list(df2.index)+list(df3.index)+list(df4.index)+list(df5.index)
print(df1.index)

Int64Index([ 881,  453, 2004, 1353,  281,  941, 1185, 1159, 1138,  599,
            ...
             834, 1730,  353, 1345, 1190, 1375,  185,  701, 1671, 1982],
           dtype='int64', length=415)


In [7]:
train = pd.concat([df1,df2,df3,df4], axis=0)
test = df5.copy()

Y_train = np.array(train['class'])
X_train= np.array(train.drop(['class'],axis=1))
# X_train=X_train.reshape(X_train.shape[0], 1 , X_train.shape[1])

Y_val=np.array(test['class'])
X_val = np.array(test.drop(['class'],axis=1))
# X_val=X_val.reshape(X_val.shape[0], 1 , X_val.shape[1])

In [8]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape,
                        activation='relu'))
    model.add(Dense(units = input_shape[0]))
    model.add(Dropout(0.1))
    model.add(Dense(units=input_shape[0]))
    model.add(Dropout(0.1))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [9]:
TIME_STEPS= X_train.shape[1]

In [10]:
model = create_RNN(hidden_units=180, dense_units=1, input_shape=(TIME_STEPS,1),
                   activation=['tanh', 'tanh'])
model.fit(X_train, Y_train, epochs=200, batch_size=64, verbose=2,validation_data=(X_val, Y_val))

Epoch 1/200
26/26 - 6s - loss: 4.0422 - val_loss: 2.1000 - 6s/epoch - 243ms/step
Epoch 2/200
26/26 - 3s - loss: 2.2024 - val_loss: 1.8229 - 3s/epoch - 114ms/step
Epoch 3/200
26/26 - 3s - loss: 1.8317 - val_loss: 2.1764 - 3s/epoch - 111ms/step
Epoch 4/200
26/26 - 4s - loss: 1.7809 - val_loss: 1.5171 - 4s/epoch - 168ms/step
Epoch 5/200
26/26 - 5s - loss: 1.4819 - val_loss: 1.4646 - 5s/epoch - 201ms/step
Epoch 6/200
26/26 - 3s - loss: 1.4715 - val_loss: 1.5417 - 3s/epoch - 103ms/step
Epoch 7/200
26/26 - 3s - loss: 1.5590 - val_loss: 1.9342 - 3s/epoch - 115ms/step
Epoch 8/200
26/26 - 4s - loss: 1.5083 - val_loss: 1.5918 - 4s/epoch - 173ms/step
Epoch 9/200
26/26 - 4s - loss: 1.4412 - val_loss: 1.4365 - 4s/epoch - 136ms/step
Epoch 10/200
26/26 - 3s - loss: 1.4795 - val_loss: 1.8294 - 3s/epoch - 127ms/step
Epoch 11/200
26/26 - 3s - loss: 1.4438 - val_loss: 1.6144 - 3s/epoch - 113ms/step
Epoch 12/200
26/26 - 5s - loss: 1.3718 - val_loss: 1.4529 - 5s/epoch - 179ms/step
Epoch 13/200
26/26 - 3s -

In [11]:
#run
import time
st = time.time()
p1 = np.array(model(X_val)).flatten()
end = time.time()
# print(end, st, len(p1))
print((end-st)/len(p1))
p2 = np.array(model(X_train)).flatten()
print(X_train.shape)
print(X_val.shape)

0.001599750174097268
(1660, 180)
(415, 180)


In [12]:
from scipy import stats
from scipy.stats import kendalltau
# print(np.corrcoef(p2, Y_train), stats.spearmanr(p2, Y_train), kendalltau(p2,Y_train).correlation)
np.corrcoef(p1, Y_val)[1][0], stats.spearmanr(p1, Y_val).correlation, kendalltau(p1,Y_val).correlation


(0.4289436965581745, 0.40809121851666413, 0.2833540246822248)